In [45]:
import numpy as np
import pandas as pd

In [46]:
def optimize_dataframe_for_memory(df):
 print("current memeor usage: ", df.memory_usage().sum() / 1024**2, "MB")

 for col in df.columns:
  _max = df[col].max()
  _min = df[col].min()
  if df[col].dtype  == 'int64':
   if _min > np.iinfo(np.int8).min and _max < np.iinfo(np.int8).max:
    df[col] = df[col].astype(np.int8)
   elif _min > np.iinfo(np.int16).min and _max < np.iinfo(np.int16).max:
    df[col] = df[col].astype(np.int16)
   elif _min > np.iinfo(np.int32).min and _max < np.iinfo(np.int32).max:
    df[col] = df[col].astype(np.int32)
  elif df[col].dtype == 'float64':
   if _min > np.finfo(np.float16).min and _max < np.finfo(np.float16).max:
    df[col] = df[col].astype(np.float16)
   elif _min > np.finfo(np.float32).min and _max < np.finfo(np.float32).max:
    df[col] = df[col].astype(np.float32)
  else:
   continue
 
 print("new memory usage: ", df.memory_usage().sum() / 1024**2, "MB")
 return df


In [47]:
data_set_location  = 'data/'

In [48]:
departments_df = pd.read_csv(data_set_location + 'departments.csv')
aisles_df = pd.read_csv(data_set_location + 'aisles.csv')


orders_df = pd.read_csv(data_set_location + 'orders.csv')
product_df = pd.read_csv(data_set_location + 'products.csv')

order_products_train_df = pd.read_csv(data_set_location + 'order_products__train.csv')
order_products_prior_df = pd.read_csv(data_set_location + 'order_products__prior.csv')


In [49]:
departments_df = optimize_dataframe_for_memory(departments_df)
aisles_df = optimize_dataframe_for_memory(aisles_df)

orders_df = optimize_dataframe_for_memory(orders_df)
product_df = optimize_dataframe_for_memory(product_df)

order_products_train_df = optimize_dataframe_for_memory(order_products_train_df)
order_products_prior_df = optimize_dataframe_for_memory(order_products_prior_df)

current memeor usage:  0.0004425048828125 MB
new memory usage:  0.00030231475830078125 MB
current memeor usage:  0.002166748046875 MB
new memory usage:  0.001399993896484375 MB
current memeor usage:  182.7056655883789 MB
new memory usage:  68.5147008895874 MB
current memeor usage:  1.5164794921875 MB
new memory usage:  0.7109146118164062 MB
current memeor usage:  42.255279541015625 MB
new memory usage:  13.204858779907227 MB
current memeor usage:  989.8221740722656 MB
new memory usage:  340.2514524459839 MB


In [50]:
product_feats_df = pd.read_pickle(data_set_location + 'product_features.pkl')
user_feats_df = pd.read_pickle(data_set_location + 'user_features.pkl')
product_user_feats_df = pd.read_pickle(data_set_location + 'user_product_features.pkl')

In [51]:
"""

combine data in ordera

"""


df_orders = pd.merge( orders_df, order_products_train_df , on='order_id', how='inner')
df_orders.drop(['eval_set', 'add_to_cart_order', 'order_id'], axis = 1, inplace = True)

df_orders.head(3)

,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,reordered
0,1,11,4,8,14.0,196,1
1,1,11,4,8,14.0,25133,1
2,1,11,4,8,14.0,38928,1


In [52]:
df_users= df_orders.user_id.unique()

In [53]:
product_user_feats_df.head(2)

,user_id,product_id,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,is_reorder_1
0,1,196,10,9,0.900000,1.400000,17.59375,10,1.0,1.0,1.0
1,1,10258,9,8,0.888889,3.333333,19.56250,10,1.0,1.0,1.0


In [54]:
df = product_user_feats_df[product_user_feats_df.user_id.isin(df_users)]

df.head(2)

,user_id,product_id,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,is_reorder_1
0,1,196,10,9,0.900000,1.400000,17.59375,10,1.0,1.0,1.0
1,1,10258,9,8,0.888889,3.333333,19.56250,10,1.0,1.0,1.0


In [55]:
df = pd.merge( df ,  df_orders, on = ['user_id', 'product_id'], how = 'outer')

In [56]:
df.columns

Index(['user_id', 'product_id', 'total_product_orders_by_user',
       'total_product_reorders_by_user', 'user_product_reorder_percentage',
       'avg_add_to_cart_by_user', 'avg_days_since_last_bought',
       'last_ordered_in', 'is_reorder_3', 'is_reorder_2', 'is_reorder_1',
       'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'reordered'],
      dtype='object')

In [57]:
# Impute null values for order features:
# - order_number
# - order_dow (day of week)
# - order_hour_of_day
# - days_since_prior_order
# 
# Use mean values grouped by users to fill nulls.
# Rationale: These products with missing data are still 
# potential candidates for orders, so we estimate 
# their values based on user purchasing patterns.

In [58]:
df.order_number.fillna(df.groupby('user_id')['order_number'].transform('mean'), inplace = True)
df.order_dow.fillna(df.groupby('user_id')['order_dow'].transform('mean'), inplace = True)
df.order_hour_of_day.fillna(df.groupby('user_id')['order_hour_of_day'].transform('mean'), inplace = True)
df.days_since_prior_order.fillna(df.groupby('user_id')['days_since_prior_order'].transform('mean'), inplace = True)

In [59]:
df.columns

Index(['user_id', 'product_id', 'total_product_orders_by_user',
       'total_product_reorders_by_user', 'user_product_reorder_percentage',
       'avg_add_to_cart_by_user', 'avg_days_since_last_bought',
       'last_ordered_in', 'is_reorder_3', 'is_reorder_2', 'is_reorder_1',
       'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'reordered'],
      dtype='object')

In [60]:
df.reordered.isnull().sum()

7645837

In [61]:
df = df[df.reordered != 0]

In [62]:
df.reordered.fillna(0, inplace = True)

In [63]:
df.reordered.isnull().sum()

0

In [64]:
df.head(2)

,user_id,product_id,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,is_reorder_1,order_number,order_dow,order_hour_of_day,days_since_prior_order,reordered
0,1,196,10.0,9.0,0.900000,1.400000,17.59375,10.0,1.0,1.0,1.0,11.0,4.0,8.0,14.0,1.0
1,1,10258,9.0,8.0,0.888889,3.333333,19.56250,10.0,1.0,1.0,1.0,11.0,4.0,8.0,14.0,1.0


In [65]:
df = pd.merge( df, product_feats_df, on = 'product_id', how = 'left')
df = pd.merge( df, user_feats_df, on = 'user_id', how = 'left')
df.head(2)

,user_id,product_id,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,...,reordered_average_order_size_x,reordered_reorder_in_order_x,reordered_average_order_size_y,reordered_reorder_in_order_y,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,196,10.0,9.0,0.900000,1.400000,17.59375,10.0,1.0,1.0,...,5.9,0.705833,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
1,1,10258,9.0,8.0,0.888889,3.333333,19.56250,10.0,1.0,1.0,...,5.9,0.705833,5.9,0.705833,6,6,9,0.666667,1.0,0.666667


In [66]:
df.shape

(8474661, 70)

In [67]:
df.to_pickle(data_set_location + 'final_data.pkl')